# Define Paths

In [70]:
from pathlib import Path
source_path = Path("W:/test-source")
dest_path = Path("W:/test-dest")


In [96]:
import filecmp
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
import shutil

# ensure destination folder exists.
dest_path.mkdir(parents=True, exist_ok=True)
# get all files.
total_moved_gb = 0
total_moved_files = 0
for source_file in source_path.rglob('*'):
    # if this is a folder check if its empty.
    if source_file.is_dir():
        if not any(source_file.iterdir()):
            source_file.rmdir()
    # check if file.
    if source_file.is_file():
        # Get relative file path.
        relative_file_path = source_file.relative_to(source_path)
        destination_file = dest_path/relative_file_path
        # check if file already exists.
        if destination_file.is_file():
            # check if files are the same size.
            if filecmp.cmp(source_file,destination_file,shallow=True):
                # delete source file.
                remove_file = True
                move_file = False
            else:
                remove_file = False
                move_file = True
        # file not yet present
        else:
            remove_file = False
            move_file = True
        # copy file.
        if move_file:
            # check if directory exists.
            destination_file.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(source_file,destination_file)
            # check succesfull copy.
            if filecmp.cmp(source_file,destination_file,shallow=True):
                remove_file = True
            else:
                remove_file = False
    if remove_file:
        total_moved_files+=1
        file_size_gb = source_file.stat().st_size*10**-9
        total_moved_gb += file_size_gb
        source_file.unlink()
        clear_output(wait=True)
        display(f"Total moved: {total_moved_files} files, {total_moved_gb:.3f} GB")
        display(f"Last file: {relative_file_path} ({file_size_gb:.3f} GB)")

'Total moved: 5 files, 0.524 GB'

'Last file: 4.dummy (0.105 GB)'

# Appendix
## Create test data

In [15]:
def create_dummy_file(file_loc,file_size_mb):
    with open(file_loc, 'wb') as f:
        f.seek(file_size_mb * 1024 * 1024) # One GB
        f.write(b'\x21')

### Create 5 dummy files in source

In [95]:
for i in range(5):
    create_dummy_file(source_path/f"{i}.dummy",100)

In [66]:
for i in range(5):
    create_dummy_file(dest_path/f"{i}.dummy",50)